<a href="https://colab.research.google.com/github/oluwafemidiakhoa/MLprject/blob/main/SmartCityAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU langchain-core==0.2.9
!pip install -qU langchain-community==0.2.5
!pip install -qU 'crewai[tools]'==0.32.0
!pip install -qU langchain-groq==0.1.5


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.6/987.6 kB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.5/366.5 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.5/68.5 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.7/194.7 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

## Step 2: Import Required Dependencies
Import the necessary libraries and modules.

In [2]:
import json
import os
import sqlite3
from dataclasses import asdict, dataclass
from datetime import datetime, timezone
from pathlib import Path
from textwrap import dedent
from typing import Any, Dict, List, Tuple, Union

import pandas as pd
from crewai import Agent, Crew, Process, Task
from crewai_tools import tool
from google.colab import userdata
from langchain.schema import AgentFinish
from langchain.schema.output import LLMResult
from langchain_community.tools.sql_database.tool import (
    InfoSQLDatabaseTool,
    ListSQLDatabaseTool,
    QuerySQLCheckerTool,
    QuerySQLDataBaseTool,
)
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_core.callbacks.base import BaseCallbackHandler
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

# Setup API key for GROQ
os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")


## Step 3: Prepare the Data
Load your data into a Pandas DataFrame and save it to an SQLite database. The data will include information from various sensors (e.g., traffic sensors, energy meters, emergency response units).

In [3]:
# Example data for sensors in the smart city
traffic_data = {
    "sensor_id": [1, 2, 3, 4, 5],
    "location": ["A", "B", "C", "D", "E"],
    "traffic_flow": [200, 150, 300, 100, 250],
    "timestamp": ["2023-07-14 08:00:00", "2023-07-14 08:05:00", "2023-07-14 08:10:00", "2023-07-14 08:15:00", "2023-07-14 08:20:00"]
}

energy_data = {
    "meter_id": [1, 2, 3, 4, 5],
    "location": ["A", "B", "C", "D", "E"],
    "energy_consumption": [1500, 1200, 1800, 1100, 1600],
    "timestamp": ["2023-07-14 08:00:00", "2023-07-14 08:05:00", "2023-07-14 08:10:00", "2023-07-14 08:15:00", "2023-07-14 08:20:00"]
}

emergency_data = {
    "unit_id": [1, 2, 3, 4, 5],
    "location": ["A", "B", "C", "D", "E"],
    "response_time": [5, 6, 4, 7, 5],
    "timestamp": ["2023-07-14 08:00:00", "2023-07-14 08:05:00", "2023-07-14 08:10:00", "2023-07-14 08:15:00", "2023-07-14 08:20:00"]
}

# Convert to DataFrames
df_traffic = pd.DataFrame(traffic_data)
df_energy = pd.DataFrame(energy_data)
df_emergency = pd.DataFrame(emergency_data)

# Save DataFrames to SQLite database
connection = sqlite3.connect("smart_city.db")
df_traffic.to_sql(name="traffic", con=connection, if_exists='replace')
df_energy.to_sql(name="energy", con=connection, if_exists='replace')
df_emergency.to_sql(name="emergency", con=connection, if_exists='replace')


5

## Step 4: Setup the LLM
Configure the LLM using ChatGroq.

In [4]:
@dataclass
class Event:
    event: str
    timestamp: str
    text: str

def _current_time() -> str:
    return datetime.now(timezone.utc).isoformat()

class LLMCallbackHandler(BaseCallbackHandler):
    def __init__(self, log_path: Path):
        self.log_path = log_path

    def on_llm_start(self, serialized: Dict[str, Any], prompts: List[str], **kwargs: Any) -> Any:
        assert len(prompts) == 1
        event = Event(event="llm_start", timestamp=_current_time(), text=prompts[0])
        with self.log_path.open("a", encoding="utf-8") as file:
            file.write(json.dumps(asdict(event)) + "\n")

    def on_llm_end(self, response: LLMResult, **kwargs: Any) -> Any:
        generation = response.generations[-1][-1].message.content
        event = Event(event="llm_end", timestamp=_current_time(), text=generation)
        with self.log_path.open("a", encoding="utf-8") as file:
            file.write(json.dumps(asdict(event)) + "\n")

llm = ChatGroq(
    temperature=0,
    model_name="llama3-70b-8192",  # or another model name if applicable
    callbacks=[LLMCallbackHandler(Path("prompts.jsonl"))],
)


## Step 5: Create Tools
Define the tools that the agents will use to interact with the SQL database.

In [6]:
# Establish a database connection
db = SQLDatabase.from_uri("sqlite:///smart_city.db")

# Tool to list all tables in the database
@tool("list_tables")
def list_tables() -> str:
    """
    Lists all tables in the database.
    """
    return ListSQLDatabaseTool(db=db).invoke("")

# Tool to return the schema and sample rows for given tables
@tool("tables_schema")
def tables_schema(tables: str) -> str:
    """
    Returns the schema and sample rows for the given tables.

    Args:
        tables (str): A comma-separated string of table names.

    Returns:
        str: Schema and sample rows for the specified tables.
    """
    tool = InfoSQLDatabaseTool(db=db)
    return tool.invoke(tables)

# Tool to execute a given SQL query
@tool("execute_sql")
def execute_sql(sql_query: str) -> str:
    """
    Executes a given SQL query and returns the result.

    Args:
        sql_query (str): The SQL query to execute.

    Returns:
        str: The result of the executed SQL query.
    """
    return QuerySQLDataBaseTool(db=db).invoke(sql_query)

# Tool to check the SQL query before executing it
@tool("check_sql")
def check_sql(sql_query: str) -> str:
    """
    Checks the SQL query for errors before executing it.

    Args:
        sql_query (str): The SQL query to check.

    Returns:
        str: The result of the SQL query check.
    """
    return QuerySQLCheckerTool(db=db, llm=llm).invoke({"query": sql_query})


## Step 6: Create Agents
Define the agents with specific roles and tools.

In [7]:
# Traffic Management Agent
traffic_agent = Agent(
    role="Traffic Management Specialist",
    goal="Monitor and optimize traffic flow based on sensor data",
    backstory=dedent("""
        You are responsible for ensuring smooth traffic flow in the city. Use sensor data to identify congestion and suggest optimization measures.
    """),
    llm=llm,
    tools=[list_tables, tables_schema, execute_sql, check_sql],
    allow_delegation=False,
)

# Energy Management Agent
energy_agent = Agent(
    role="Energy Management Specialist",
    goal="Monitor and optimize energy usage based on meter data",
    backstory=dedent("""
        You ensure efficient energy usage throughout the city. Analyze meter data to identify high consumption areas and suggest energy-saving measures.
    """),
    llm=llm,
    tools=[list_tables, tables_schema, execute_sql, check_sql],
    allow_delegation=False,
)

# Emergency Response Agent
emergency_agent = Agent(
    role="Emergency Response Coordinator",
    goal="Monitor and improve emergency response times",
    backstory=dedent("""
        You coordinate emergency response units to ensure quick response times. Use data to identify bottlenecks and suggest improvements.
    """),
    llm=llm,
    tools=[list_tables, tables_schema, execute_sql, check_sql],
    allow_delegation=False,
)


## Step 7: Create Tasks
Define the tasks for monitoring and optimizing various city systems.

In [8]:
# Task to monitor and analyze traffic flow
monitor_traffic = Task(
    description="Monitor and analyze traffic flow based on sensor data.",
    expected_output="Traffic flow analysis and optimization suggestions",
    agent=traffic_agent,
)

# Task to monitor and analyze energy usage
monitor_energy = Task(
    description="Monitor and analyze energy usage based on meter data.",
    expected_output="Energy usage analysis and optimization suggestions",
    agent=energy_agent,
)

# Task to monitor and analyze emergency response times
monitor_emergency = Task(
    description="Monitor and analyze emergency response times.",
    expected_output="Emergency response time analysis and optimization suggestions",
    agent=emergency_agent,
)


## Step 8: Setup the Crew
Create a Crew instance to manage the agents and tasks.

In [9]:
crew = Crew(
    agents=[traffic_agent, energy_agent, emergency_agent],
    tasks=[monitor_traffic, monitor_energy, monitor_emergency],
    process=Process.sequential,
    verbose=2,
    memory=False,
    output_log_file="smart_city_crew.log",
)


## Step 9: Kickoff the Crew for Monitoring
Start the monitoring process for various city systems.

python


In [10]:
# Monitor traffic
inputs = {
    "query": "Analyze traffic flow and suggest optimizations."
}
result = crew.kickoff(inputs=inputs)
print(result)

# Monitor energy usage
inputs = {
    "query": "Analyze energy usage and suggest optimizations."
}
result = crew.kickoff(inputs=inputs)
print(result)

# Monitor emergency response times
inputs = {
    "query": "Analyze emergency response times and suggest optimizations."
}
result = crew.kickoff(inputs=inputs)
print(result)


 [2024-07-16 14:59:34][DEBUG]: == Working Agent: Traffic Management Specialist
 [2024-07-16 14:59:34][INFO]: == Starting Task: Monitor and analyze traffic flow based on sensor data.
 

emergency, energy, traffic

 


CREATE TABLE traffic (
	"index" INTEGER, 
	sensor_id INTEGER, 
	location TEXT, 
	traffic_flow INTEGER, 
	timestamp TEXT
)

/*
3 rows from traffic table:
index	sensor_id	location	traffic_flow	timestamp
0	1	A	200	2023-07-14 08:00:00
1	2	B	150	2023-07-14 08:05:00
2	3	C	300	2023-07-14 08:10:00
*/

 

[('A', 200.0), ('B', 150.0), ('C', 300.0), ('D', 100.0), ('E', 250.0)]

 

[('C', 300.0), ('E', 250.0), ('A', 200.0), ('B', 150.0), ('D', 100.0)]

 

Error: (sqlite3.OperationalError) misuse of aggregate: AVG()
[SQL: SELECT location, AVG(traffic_flow) as avg_traffic_flow FROM traffic WHERE avg_traffic_flow > 200 GROUP BY location]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

 

[('C', 300.0), ('E', 250.0)]

 [2024-07-16 15:00:17][DEBUG]: == [Traffic Management Spe